In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv("data/AB_NYC_2019.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/AB_NYC_2019.csv'

In [ ]:
data.shape

In [ ]:
#data.groupby('host_id').count().max()

In [ ]:
#neighbourhood_group to first

data['id'] = data.index
data = data.set_index('neighbourhood_group')
data.head()

## 設定Matplotlib樣式

In [ ]:
# 可使用的樣式
print(plt.style.available)

In [ ]:
#設定樣式
plt.style.use('Solarize_Light2')

## 哪個區域的案件最多

In [ ]:
data.groupby('neighbourhood_group').size()

In [ ]:
data.groupby('neighbourhood').size()

In [ ]:
# sns.countplot(x='neighbourhood_group', data=data)

# plt.xlabel('Zone')
# plt.ylabel('Case Number')

## 哪個區域的平均價格最高

In [ ]:
#NG_price_s = data.groupby(by=['neighbourhood_group'])['price'].sum() 
#NG_price_c = data.groupby(by=['neighbourhood_group'])['price'].count() 
#NG_price = NG_price_s / NG_price_c

#same as
#data.groupby('neighbourhood_group')['price'].mean()
data.groupby('neighbourhood_group')['price'].mean().reset_index()  #reset_index()製成表格

In [ ]:
# 如何把輸出資料作圖 Ans

sns.barplot(x='neighbourhood_group', 
            y='price',
            data=data.groupby('neighbourhood_group')['price'].mean().reset_index())

plt.xlabel('Zone')
plt.ylabel('Avg Price')

## 畫出不同區域各種房型的平均價格

In [ ]:
# 如何做 區域/房型/價格的總和 （三位）Ans

avg_price = data.groupby(['neighbourhood_group','room_type'])['price'].mean().reset_index()
avg_price 

In [ ]:
sns.barplot(x='neighbourhood_group', 
            y='price', 
            hue='room_type',
            data=avg_price )

plt.xlabel('Zone')
plt.ylabel('Avg Price')
plt.legend(title='Room Type')

In [ ]:
sns.barplot(x='room_type', 
            y='price', 
            hue='neighbourhood_group',
            data=data.groupby(['neighbourhood_group','room_type'])['price'].mean().reset_index())

plt.xlabel('Room Type')
plt.ylabel('Avg Price')
plt.legend(title='Zone')

## 擁有房間數目最多的前10名房東

In [ ]:
# 不知道為何不能這樣用 會 merge 不到資料
# -> 
data_index = data['host_id'].value_counts().head(3).reset_index()
data_index

In [ ]:
#data_index = data.groupby('host_id').size().sort_values(ascending=False).head(3).reset_index()
#data_index

In [ ]:
# 不知道如何把 host_id 

# pd.merge(data_index,
#         data,
#         on='host_id')

# same host_id dropout
pd.merge(data_index,
         data[['host_id','host_name']].drop_duplicates('host_id'),
         on='host_id')

## 擁有房間租金總額最多的前10名房東

In [ ]:
data_index = data.groupby('host_id')['price'].sum().sort_values(ascending=False).head(10).reset_index()
data_index

In [ ]:
pd.merge(data_index,data[['host_id','host_name']].drop_duplicates('host_id'),
         on='host_id')

## 那個區域比較熱門

In [ ]:
data.groupby('neighbourhood_group')['number_of_reviews'].sum().sort_values(ascending=False).reset_index()

In [ ]:
data.groupby('neighbourhood')['number_of_reviews'].sum().sort_values(ascending=False).head(10).reset_index()

In [ ]:
subData = data.groupby(['neighbourhood_group','neighbourhood'])['number_of_reviews']\
    .sum().sort_values(ascending=False).head(10).reset_index()
subData

In [ ]:
g = sns.barplot(data=subData,
                x='neighbourhood',
                y='number_of_reviews')

plt.xticks(rotation=90)
i = 0
for value in subData['number_of_reviews'].values:
    g.text(i, 7000, 
           value, 
           color='#0000FF', size=14, rotation= 90, ha="center")
    i += 1
    

In [ ]:
data = pd.read_csv("data/AB_NYC_2019.csv")



plt.figure(figsize=(10,6))
sns.scatterplot(x=data.longitude,
                y=data.latitude,
                hue=data.neighbourhood_group)

## 地理圖

## == Folium == 

#pip install folium
https://python-visualization.github.io/folium/


In [ ]:
pip install folium

In [ ]:
#pip install folium

import folium
from folium.plugins import HeatMap
m=folium.Map([40.7128,- 73.9160],  #define a location
             zoom_start=11)       #zoom more

HeatMap(data[['latitude','longitude']].dropna(),
        radius=8,
        gradient={0.2:'blue',0.4:'purple',0.6:'orange',1.0:'red'}).add_to(m)
display(m)

In [ ]:
import folium
from folium.plugins import HeatMap
d = data.head(100)
m=folium.Map([40.7128,-74.0060],zoom_start=11)
for index, row in d.iterrows():
    folium.Marker(row[['latitude','longitude']], popup=row['price']).add_to(m)
display(m)

In [ ]:
import folium
from folium import plugins
d = data.head(10000)
m=folium.Map([40.7128,-74.0060],zoom_start=11)
mc=plugins.MarkerCluster().add_to(m)
for index, row in d.iterrows():
    folium.Marker(row[['latitude','longitude']], popup=row['price']).add_to(mc)
display(m)

## == sns.displot ==
#pip install seaborn

In [ ]:
pip install --upgrade seaborn

In [ ]:
# import seaborn as sns
# sns.displot(data[data['price']<2000],
#             x='price',
#             hue='neighbourhood_group',
#             bins=20)

In [ ]:
plt.figure(figsize=(10,6))
sns.distplot(data[data.neighbourhood_group=='Manhattan'].price,color='maroon',hist=False,label='Manhattan')
sns.distplot(data[data.neighbourhood_group=='Brooklyn'].price,color='black',hist=False,label='Brooklyn')
sns.distplot(data[data.neighbourhood_group=='Queens'].price,color='green',hist=False,label='Queens')
sns.distplot(data[data.neighbourhood_group=='Staten Island'].price,color='blue',hist=False,label='Staten Island')
sns.distplot(data[data.neighbourhood_group=='Long Island'].price,color='lavender',hist=False,label='Long Island')
plt.title('Borough wise price destribution for price<2000')
plt.xlim(0,2000)
plt.show()